In [1]:
import pickle as pkl

import polars as pl

from backtest_lib.market.polars_impl import PolarsPastView

close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns
close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_dataframe(close_prices_df)

past_cost_prices

PolarsPastView(_by_security=PolarsBySecurity(_security_column_df=shape: (1_486, 488)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ A         ┆ AMZN      ┆ ABT       ┆ AAPL      ┆ … ┆ VMC       ┆ USB       ┆ V         ┆ TRV      │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 85.949997 ┆ 94.900497 ┆ 86.949997 ┆ 75.087502 ┆ … ┆ 142.72000 ┆ 59.200001 ┆ 191.11999 ┆ 137.5099 │
│           ┆           ┆           ┆           ┆   ┆ 1         ┆           ┆ 5         ┆ 95       │
│ 84.57     ┆ 93.748497 ┆ 85.889999 ┆ 74.357498 ┆ … ┆ 142.10000 ┆ 58.509998 ┆ 189.60000 ┆ 137.0200 │
│           ┆           ┆           ┆           ┆   ┆ 6         ┆           ┆ 6         ┆ 04       │
│ 84.8

In [2]:
from backtest_lib.market import MarketView, PastUniversePrices
from backtest_lib.market.polars_impl import PolarsPastView

df = past_cost_prices.by_security.as_df(show_periods=False)
cols = df.columns
ewma = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_past_view = PolarsPastView.from_dataframe(ewma)

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"ewma_ratio": ewma_past_view},
)

market_view.signals["ewma_ratio"].by_period[-1]

SeriesUniverseMapping(names=('A', 'AMZN', 'ABT', 'AAPL', 'ADM', 'AMP', 'T', 'AON', 'AIG', 'ADP', 'MMM', 'AJG', 'ABBV', 'ACGL', 'ANET', 'AXP', 'AES', 'AEP', 'AWK', 'ADBE', 'ALLE', 'APH', 'AMAT', 'APD', 'AMD', 'APTV', 'MO', 'AFL', 'APA', 'ARE', 'GOOGL', 'AKAM', 'ADI', 'GOOG', 'AIZ', 'AMT', 'AEE', 'AMGN', 'ALGN', 'ALL', 'AMCR', 'AOS', 'APO', 'ADSK', 'LNT', 'ATO', 'ACN', 'ALB', 'AME', 'CNC', 'CBRE', 'XYZ', 'TECH', 'AVB', 'BR', 'BX', 'BXP', 'AVGO', 'BA', 'CPB', 'CZR', 'BKR', 'BK', 'BF-B', 'CVX', 'CF', 'CDNS', 'BIIB', 'CNP', 'AXON', 'CPT', 'CHTR', 'CCL', 'AZO', 'BMY', 'CAT', 'SCHW', 'CRL', 'BRO', 'CDW', 'BG', 'KMX', 'CHRW', 'CBOE', 'BRK-B', 'COF', 'AVY', 'BLDR', 'BAX', 'BKNG', 'CAH', 'BLK', 'COR', 'BAC', 'BSX', 'BDX', 'BBY', 'BALL', 'DDOG', 'FSLR', 'DAL', 'CFG', 'CMI', 'CTAS', 'CLX', 'CB', 'CINF', 'CL', 'CVS', 'CCI', 'DG', 'CTVA', 'DVA', 'DHR', 'DECK', 'C', 'DLTR', 'KO', 'CPAY', 'CTSH', 'DRI', 'GLW', 'COP', 'COST', 'CSX', 'CMS', 'DELL', 'CMG', 'CME', 'DXCM', 'CSGP', 'CMCSA', 'CSCO', 'DAY', '

In [ ]:
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy.decision import TargetWeightsDecision as Decision


def ewma_strategy(universe, current_portfolio, market, ctx):
    latest_ewma_ratio = (
        market.signals["ewma_ratio"].by_security[list(universe)].by_period[-1]
    )

    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return Decision(target=WeightedPortfolio(cash=0, holdings=weights))


decision = ewma_strategy(["AAPL", "MSFT"], None, market_view, None)
decision

Decision(target=<backtest_lib.portfolio.WeightedPortfolio object at 0x7f53441a0830>, notes=None)

In [4]:
import polars as pl

from backtest_lib.backtest import Backtest
from backtest_lib.market.polars_impl import SeriesUniverseMapping

universe = tuple(past_cost_prices.by_security.as_df(show_periods=False).columns)
pf = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe,
        pl.Series([1 / len(universe)] * len(universe)),
    ),
)
backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

total return: 88.08%


In [5]:
def index_strategy(universe, current_portfolio, market, ctx) -> Decision:
    return Decision(pf)


benchmark_backtest = Backtest(
    universe=universe,
    strategy=index_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = benchmark_backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

total return: 3.50%


In [6]:
import polars as pl

first_prices = past_cost_prices.by_period[0]
last_prices = past_cost_prices.by_period[-1]

avg_first_change = (
    past_cost_prices.by_period[1] / past_cost_prices.by_period[0]
).mean()

print("avg_first_change:", avg_first_change)

(last_prices / first_prices).mean()

avg_first_change: 0.9953024381038528


2.0048595235612363

In [7]:
aapl_prices = past_cost_prices.by_security["AAPL"].as_series()

print(aapl_prices.pct_change() + 1)
print((aapl_prices.pct_change() + 1).cum_prod())

shape: (1_486,)
Series: 'AAPL' [f64]
[
	null
	0.990278
	1.007968
	0.995297
	1.016086
	…
	1.019681
	1.016317
	1.003805
	1.002094
	1.004684
]
shape: (1_486,)
Series: 'AAPL' [f64]
[
	null
	0.990278
	0.998169
	0.993474
	1.009456
	…
	3.615648
	3.674646
	3.68863
	3.696354
	3.713667
]


In [8]:
past_cost_prices.by_period[-1].mean()

227.7998563442074

In [9]:
df = past_cost_prices.by_security.as_df(show_periods=False)
cols = df.columns
ewma_weights = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM").alias(col)) for col in cols)
    .select(cols)
    .with_columns(pl.mean_horizontal(cols).alias("avg_ratio"))
    .with_columns(((pl.col(col) / pl.col("avg_ratio")) / len(cols)) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_weights_past_view = PolarsPastView.from_dataframe(ewma_weights)
market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"precomputed_weights": ewma_weights_past_view},
)


def fast_ewma_strategy(universe, current_portfolio, market, ctx) -> Decision:
    return Decision(
        WeightedPortfolio(
            cash=0,
            holdings=market_view.signals["precomputed_weights"].by_period[-1],
        ),
    )


fast_ewma_backtest = Backtest(
    universe=universe,
    strategy=fast_ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

In [10]:
fast_ewma_backtest.run()

BacktestResults(initial_capital=1.0, portfolio_returns=[0.0, -0.00473058596754913, 0.001333974260764599, -0.0007053389183961481, 0.0033114882081805015, 0.005186055689383469, -0.002203797334549958, 0.007884001181660506, 0.0006761101347025684, 0.00235702193450167, 0.009375610183146714, 0.002234030443868381, -0.00375399668623355, 8.598788096591311e-06, 0.00262377033619337, -0.009544583385029991, -0.014957178392149975, 0.009565620151663982, -0.0025969939589628073, 0.0024595451566003495, -0.018181954397477045, 0.00625959480909205, 0.015241202889806502, 0.011088470353070526, -0.0005451257485621752, -0.006594479383779699, 0.0059954611674879945, 0.00641111262893405, 0.006416164000422586, -0.0002815850019395721, 0.0016406310310246247, -0.002444550144677092, 0.0047155145031196775, -0.0005234328383868678, -0.009137670772742825, -0.029789170279657124, -0.03268337302862302, -0.009133888429076921, -0.03867098323338006, -0.012139542715469735, 0.038745308563880004, -0.02373349829304534, 0.038610027375

In [11]:
(
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(pl.mean_horizontal(cols).alias("avg_ratio"))
    .with_columns(((pl.col(col) / pl.col("avg_ratio")) / len(cols)) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

A,AMZN,ABT,AAPL,ADM,AMP,T,AON,AIG,ADP,MMM,AJG,ABBV,ACGL,ANET,AXP,AES,AEP,AWK,ADBE,ALLE,APH,AMAT,APD,AMD,APTV,MO,AFL,APA,ARE,GOOGL,AKAM,ADI,GOOG,AIZ,AMT,AEE,…,TRMB,XEL,VTRS,WRB,WFC,ULTA,URI,WBD,UBER,WMB,ZBRA,WYNN,WTW,UPS,VZ,WST,WMT,VRSN,YUM,GWW,WAB,VRSK,WY,VST,WAT,VICI,ZTS,UNH,VRTX,VTR,DIS,WELL,VMC,USB,V,TRV,date
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns]
0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,…,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,0.002049,2020-01-02 00:00:00
0.00204,0.002043,0.002043,0.002045,0.002052,0.002043,0.002057,0.00205,0.002047,0.002051,0.002046,0.002051,0.002045,0.002052,0.002037,0.002045,0.002044,0.002052,0.002058,0.002047,0.002043,0.002045,0.00204,0.002035,0.002045,0.002034,0.002059,0.002047,0.002064,0.002058,0.002049,0.002049,0.002039,0.002049,0.002057,0.002054,0.002053,…,0.002045,0.002057,0.002035,0.002054,0.002048,0.002039,0.002039,0.002048,0.002063,0.002057,0.002043,0.002041,0.002053,0.002053,0.002044,0.002055,0.002046,0.00207,0.002051,0.002043,0.002047,0.002061,0.002055,0.002062,0.002039,0.002054,0.002053,0.002045,0.002048,0.002063,0.002044,0.002067,0.00205,0.002044,0.002047,0.00205,2020-01-03 00:00:00
0.002046,0.002061,0.00205,0.002054,0.002041,0.002048,0.002053,0.002051,0.002047,0.00205,0.002047,0.002054,0.002054,0.002054,0.002052,0.002041,0.002057,0.002053,0.002051,0.002053,0.002042,0.00204,0.002019,0.00204,0.002041,0.00202,0.002058,0.002044,0.002053,0.002055,0.002076,0.002052,0.002029,0.002074,0.002053,0.00205,0.002051,…,0.00204,0.00205,0.002076,0.002049,0.002041,0.002046,0.00204,0.002045,0.002062,0.002066,0.002053,0.002041,0.002051,0.002045,0.002043,0.002052,0.002044,0.002069,0.002048,0.002035,0.002042,0.002057,0.002048,0.002036,0.002032,0.002039,0.002042,0.002053,0.002077,0.002066,0.002039,0.002074,0.002046,0.00203,0.002044,0.00205,2020-01-06 00:00:00
0.002052,0.002059,0.002044,0.002048,0.002031,0.002035,0.002057,0.002041,0.002042,0.002036,0.002044,0.00204,0.002046,0.002043,0.002064,0.002039,0.002059,0.002053,0.002044,0.002051,0.002032,0.002051,0.002067,0.00205,0.002042,0.002018,0.002049,0.002036,0.002342,0.002014,0.002064,0.002088,0.002066,0.002064,0.002044,0.002025,0.002056,…,0.00204,0.002048,0.002083,0.002028,0.002035,0.002058,0.002043,0.002052,0.002104,0.00206,0.002045,0.002051,0.002049,0.002046,0.002033,0.002049,0.002036,0.002064,0.002052,0.002043,0.002046,0.002065,0.002042,0.00205,0.002056,0.00204,0.00205,0.002045,0.002065,0.00205,0.002044,0.002057,0.002035,0.002027,0.002044,0.002033,2020-01-07 00:00:00
0.002059,0.002042,0.002047,0.002064,0.00202,0.002048,0.002054,0.002045,0.002056,0.002049,0.002061,0.002041,0.002052,0.002029,0.002067,0.00206,0.002049,0.002044,0.002047,0.002063,0.00204,0.002049,0.002055,0.002052,0.00203,0.002052,0.002058,0.002041,0.002224,0.00204,0.002063,0.002085,0.002067,0.002064,0.002047,0.002041,0.002047,…,0.002045,0.002043,0.002069,0.002041,0.00204,0.002075,0.002039,0.002048,0.002121,0.002032,0.001999,0.002054,0.002046,0.00205,0.002038,0.002028,0.002033,0.00206,0.002049,0.002055,0.002021,0.002066,0.002045,0.002062,0.002019,0.002047,0.002043,0.002069,0.002095,0.00204,0.00204,0.00205,0.002046,0.002029,0.002063,0.002048,2020-01-08 00:00:00
…,…,…,…,…,…,…,…,…,…,